In [1]:
# Import libraries
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

from utilities import data_basic_utility as databasic
from utilities import dataframe_utility as dfutil
import features_utility as featutil

## File Details - Light GBM Regression tuning with Optuna


In [2]:
filePrefix = "A3_064_lgbm_regression_tuning"
baseDataDir = "C:/Development/Data/COSC2670/Assignment3/A3data/"
subrunDir = "subruns/"
writeSubRunFile = False
seed = databasic.get_random_seed()

In [3]:
trainFilePath = baseDataDir + 'train.tsv'
valiFilePath = baseDataDir + 'val.tsv'
featuresFilePath = baseDataDir + 'features.tsv'
testFilePath = baseDataDir + 'test.tsv'

# trainFilePath = baseDataDir + 'train_200k.tsv'
# valiFilePath = baseDataDir + 'vali_200k.tsv'
# featuresFilePath = baseDataDir + 'features_200k.tsv'
# testFilePath = baseDataDir + 'test_200k.tsv'

In [4]:
# RowID  BeerID  ReviewerID  BeerName  BeerType  Label
# df_train = pd.read_csv(baseDataDir + 'train_500k.tsv',sep='\t',
df_train = pd.read_csv(trainFilePath,sep='\t',
                         names=['RowID','BeerID','ReviewerID',
                                  'BeerName','BeerType','rating'])
df_train.head(10)

# df_vali = pd.read_csv(baseDataDir + 'vali_500k.tsv',sep='\t',
df_vali = pd.read_csv(valiFilePath,sep='\t',
                         names=['RowID','BeerID','ReviewerID',
                                  'BeerName','BeerType','rating'])
df_vali.head(10)


,RowID,BeerID,ReviewerID,BeerName,BeerType,rating
0,22,12300,2634,Rauch Ür Bock,Rauchbier,4.0
1,27,12300,5634,Rauch Ür Bock,Rauchbier,4.5
2,28,12300,3544,Rauch Ür Bock,Rauchbier,4.5
3,40,12300,6521,Rauch Ür Bock,Rauchbier,4.0
4,43,12300,10177,Rauch Ür Bock,Rauchbier,4.5
5,48,12300,2907,Rauch Ür Bock,Rauchbier,3.5
6,49,12300,1532,Rauch Ür Bock,Rauchbier,4.0
7,50,12300,3452,Rauch Ür Bock,Rauchbier,3.5
8,59,12300,6861,Rauch Ür Bock,Rauchbier,4.0
9,64,6699,6401,Caldera Pale Ale,American Pale Ale (APA),4.5


In [5]:
print(df_train.shape)
df_train.sort_values("ReviewerID").head(10)


(845008, 6)


,RowID,BeerID,ReviewerID,BeerName,BeerType,rating
167343,311098,8808,1,Samuel Adams Hallertau Imperial Pilsner,American Double / Imperial Pilsner,2.5
723418,1356865,2117,1,Censored,American Amber / Red Ale,3.0
460327,860477,4351,1,New Albanian Saison,Saison / Farmhouse Ale,4.0
59162,111627,2077,1,Coopers Extra Strong Vintage Ale,English Strong Ale,4.0
59267,111809,5108,1,Heritage Premium Ale,English Pale Ale,3.5
344194,642829,6271,1,Maharaja,American Double / Imperial IPA,4.5
39938,75813,9541,1,Sierra Nevada Southern Hemisphere Harvest Fres...,American IPA,4.5
613280,1149025,151,1,Fuller's ESB,Extra Special / Strong Bitter (ESB),4.5
197540,369886,25,1,Blanche De Chambly,Witbier,3.0
218928,409576,5232,1,Tröegs Nugget Nectar,American Amber / Red Ale,4.0


In [6]:
# Tinkering to get it working

# df_train_reviewer_counts = df_train.groupby("ReviewerID", as_index=False).size()
# df_train_reviewer_counts.columns=["ReviewerID", "ReviewerReviewCount"]

# print(type(df_train_reviewer_counts))

# df_train = pd.merge(df_train, df_train_reviewer_counts, how="inner", left_on="ReviewerID", right_on="ReviewerID")
# df_train.head()

# df_train["ReviewerReviewCount"] = df_train.groupby("ReviewerID").size()
# print(df_train.shape)
# df_train.head(20)

In [7]:
df_train = featutil.addReviewerReviewCount(df_train)
df_train = featutil.addBeerReviewCount(df_train)

df_vali = featutil.addReviewerReviewCount(df_vali)
df_vali = featutil.addBeerReviewCount(df_vali)

# df_train.head(10)

In [8]:
# RowID BrewerID ABV DayofWeek Month DayofMonth Year TimeOfDay Gender Birthday Text Lemmatized POS_Tag
# df_features = pd.read_csv(baseDataDir + 'features_500k.tsv',sep='\t', names=['RowID','BrewerID','ABV','DayofWeek','Month',
df_features = pd.read_csv(featuresFilePath, sep='\t', names=['RowID','BrewerID','ABV','DayofWeek','Month',
                                                                 'DayofMonth','Year','TimeOfDay','Gender',
                                                                 'Birthday','Text','Lemmatized','POS_Tag'])

df_features.head()

,RowID,BrewerID,ABV,DayofWeek,Month,DayofMonth,Year,TimeOfDay,Gender,Birthday,Text,Lemmatized,POS_Tag
0,18,1075,7.4,Mon,Jan,2,2012,15:20:04,Male,unknown,Pours a murky light brown with a 1 inch fizzy ...,pour a murky light brown with a 1 inch fizzy t...,VBZ DT JJ NN JJ IN DT CD NN JJ NN NN WDT VBZ I...
1,19,1075,7.4,Sun,Jan,1,2012,06:46:52,Male,unknown,Faint sudsy head with some with some dissipati...,faint sudsy head with some with some dissipate...,NN JJ NN IN DT IN DT VBG JJ NN . JJ JJ NN . DT...
2,20,1075,7.4,Tue,Nov,29,2011,05:51:44,Male,unknown,A new arrival to the West TN area ... Pours qu...,a new arrival to the West TN area ... pour qui...,"DT JJ NN IN DT NNP NNP NN , VBZ PDT DT NN JJR ..."
3,21,1075,7.4,Sat,Nov,5,2011,22:59:57,Male,unknown,Sampled 10/30/11 - Transferring the notes . A ...,sample 10/30/11 - transfer the note . a ruby p...,VBN CD HYPH VBG DT NNS . DT NN VBP IN DT NN NN...
4,22,1075,7.4,Tue,Nov,1,2011,20:40:21,Male,"Oct 14, 1983",This is my first rauchbier . Pours a burnt amb...,this be my first rauchbier . pour a burn amber...,DT VBZ PRP$ JJ NN . VBZ DT VBN NN . JJ NN . NN...


In [9]:
colsToUse = ["RowID", "BrewerID", "ABV", "DayofWeek", "DayofMonth", "Month", "Year", "Gender", "TimeOfDay", "Birthday"]

# Find the feature records that match the training and validation data and join them together
dfFullData = df_train.join(df_features[colsToUse], on="RowID", how="inner", rsuffix="Feat")
dfFullDataVali = df_vali.join(df_features[colsToUse], on="RowID", how="inner", rsuffix="Feat")

dfFullData.head()

,RowID,BeerID,ReviewerID,BeerName,BeerType,rating,ReviewerReviewCount,BeerReviewCount,RowIDFeat,BrewerID,ABV,DayofWeek,DayofMonth,Month,Year,Gender,TimeOfDay,Birthday
0,19,12300,10635,Rauch Ür Bock,Rauchbier,4.0,200,23,37,1075,7.4,Mon,23,May,2011,Male,15:36:28,unknown
1,21,12300,6547,Rauch Ür Bock,Rauchbier,4.5,10,23,40,1075,7.4,Mon,16,May,2011,Male,00:31:46,unknown
2,23,12300,9789,Rauch Ür Bock,Rauchbier,4.5,164,23,42,1075,7.4,Sun,10,Apr,2011,unknown,12:17:26,unknown
3,24,12300,7372,Rauch Ür Bock,Rauchbier,5.0,432,23,43,1075,7.4,Wed,30,Mar,2011,Male,14:08:00,unknown
4,25,12300,1302,Rauch Ür Bock,Rauchbier,4.5,500,23,45,1075,7.4,Thu,24,Mar,2011,Male,14:07:00,"Jul 25, 1984"


In [10]:
# Remove the duplicated Row ID, also remove Beer Name at this point, we're nt using it
df_train_data = dfFullData.drop(['RowIDFeat', "BeerName"],axis=1)
df_vali_data = dfFullDataVali.drop(['RowIDFeat', "BeerName"],axis=1)

# df_train_data[df_train_data["ReviewerID"] == 9789].head(20)

df_train_data.head()

,RowID,BeerID,ReviewerID,BeerType,rating,ReviewerReviewCount,BeerReviewCount,BrewerID,ABV,DayofWeek,DayofMonth,Month,Year,Gender,TimeOfDay,Birthday
0,19,12300,10635,Rauchbier,4.0,200,23,1075,7.4,Mon,23,May,2011,Male,15:36:28,unknown
1,21,12300,6547,Rauchbier,4.5,10,23,1075,7.4,Mon,16,May,2011,Male,00:31:46,unknown
2,23,12300,9789,Rauchbier,4.5,164,23,1075,7.4,Sun,10,Apr,2011,unknown,12:17:26,unknown
3,24,12300,7372,Rauchbier,5.0,432,23,1075,7.4,Wed,30,Mar,2011,Male,14:08:00,unknown
4,25,12300,1302,Rauchbier,4.5,500,23,1075,7.4,Thu,24,Mar,2011,Male,14:07:00,"Jul 25, 1984"


In [11]:
print(df_train_data.shape)
print(df_vali_data.shape)

(746207, 16)
(243834, 16)


In [12]:
num_trials = 250 # 100

colsToUse.append("BeerType")
colsToUse.append("ReviewerReviewCount")
colsToUse.append("BeerReviewCount")

considerCol = {}

# Initialize them all to True, use all the cols
for colName in colsToUse:
  considerCol[colName] = True

# "RowID", "BrewerID", "BeerType", "ABV", "DayofWeek", "DayofMonth", "Month", "Year", "Gender", "TimeOfDay"
# Disable any column we want

# Usually always use Year and Review Count Columns
# considerCol["Year"] = False
# considerCol["ReviewerReviewCount"] = False
# considerCol["BeerReviewCount"] = False

# Beer Context Columns
# considerCol["ABV"] = False
# considerCol["BrewerID"] = False
# considerCol["BeerType"] = False

# Consumer Context Columns
# considerCol["DayofWeek"] = False
# considerCol["DayofMonth"] = False
# considerCol["Month"] = False
# considerCol["TimeOfDay"] = False
# considerCol["Birthday"] = False
# considerCol["Gender"] = False


For any column set to not be considered (False), then drop the columns using del

Otherwise, depending on the column, either keep the column as is, or format where required

In [13]:
if considerCol["ABV"] == False:
  del df_train_data["ABV"]
  del df_vali_data["ABV"]

In [14]:
if considerCol["BrewerID"] == False:
  del df_train_data["BrewerID"]
  del df_vali_data["BrewerID"]
else: 
  df_train_data, df_vali_data = dfutil.getDummiesForSplitSets(df_train_data, df_vali_data, "BrewerID")


c:\Development\COSC2670\Assignment3\utilities\dataframe_utility.py:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_combined.columns = df_combined.columns.str.replace(" ", "").str.replace("/", "").str.replace("-", "") \


In [15]:
if considerCol["BeerType"] == False:
  del df_train_data["BeerType"]
  del df_vali_data["BeerType"]
else: 
  df_train_data, df_vali_data = dfutil.getDummiesForSplitSets(df_train_data, df_vali_data, "BeerType")

In [16]:
if considerCol["Gender"] == False:
  del df_train_data["Gender"]
  del df_vali_data["Gender"]
else: 
  df_train_data, df_vali_data = dfutil.getDummiesForSplitSets(df_train_data, df_vali_data, "Gender")

In [17]:
if considerCol["DayofWeek"] == False:
  del df_train_data["DayofWeek"]
  del df_vali_data["DayofWeek"]
else: 
  df_train_data = featutil.formatDayOfWeek(df_train_data)
  df_vali_data = featutil.formatDayOfWeek(df_vali_data)

In [18]:
if considerCol["Month"] == False:
  del df_train_data["Month"]
  del df_vali_data["Month"]
else: 
  df_train_data = featutil.formatMonth(df_train_data)
  df_vali_data = featutil.formatMonth(df_vali_data)

In [19]:
if considerCol["DayofMonth"] == False:
  del df_train_data["DayofMonth"]
  del df_vali_data["DayofMonth"]

In [20]:
if considerCol["Year"] == False:
  del df_train_data["Year"]
  del df_vali_data["Year"]

In [21]:
if considerCol["TimeOfDay"] == False:
  del df_train_data["TimeOfDay"]
  del df_vali_data["TimeOfDay"]
else: 
  df_train_data = featutil.formatTimeToSec(df_train_data)
  df_vali_data = featutil.formatTimeToSec(df_vali_data)

In [22]:
if considerCol["Birthday"] == False:
  del df_train_data["Birthday"]
  del df_vali_data["Birthday"]
else: 
  df_train_data = featutil.convertBirthdayToAge(df_train_data)
  df_vali_data = featutil.convertBirthdayToAge(df_vali_data)

In [23]:
if considerCol["ReviewerReviewCount"] == False:
  del df_train_data["ReviewerReviewCount"]
  del df_vali_data["ReviewerReviewCount"]

In [24]:
if considerCol["BeerReviewCount"] == False:
  del df_train_data["BeerReviewCount"]
  del df_vali_data["BeerReviewCount"]

In [25]:
print(df_train_data.shape)
print(df_vali_data.shape)

df_train_data.head()

(746207, 2196)
(243834, 2196)


,RowID,BeerID,ReviewerID,rating,ReviewerReviewCount,BeerReviewCount,ABV,DayofWeek,DayofMonth,Month,...,BeerType_SmokedBeer,BeerType_Tripel,BeerType_ViennaLager,BeerType_Weizenbock,BeerType_Wheatwine,BeerType_WinterWarmer,BeerType_Witbier,Gender_Female,Gender_Male,Gender_unknown
0,19,12300,10635,4.0,200,23,7.4,1,23,5,...,0,0,0,0,0,0,0,0,1,0
1,21,12300,6547,4.5,10,23,7.4,1,16,5,...,0,0,0,0,0,0,0,0,1,0
2,23,12300,9789,4.5,164,23,7.4,7,10,4,...,0,0,0,0,0,0,0,0,0,1
3,24,12300,7372,5.0,432,23,7.4,3,30,3,...,0,0,0,0,0,0,0,0,1,0
4,25,12300,1302,4.5,500,23,7.4,4,24,3,...,0,0,0,0,0,0,0,0,1,0


In [26]:
df_vali_data.head()

,RowID,BeerID,ReviewerID,rating,ReviewerReviewCount,BeerReviewCount,ABV,DayofWeek,DayofMonth,Month,...,BeerType_SmokedBeer,BeerType_Tripel,BeerType_ViennaLager,BeerType_Weizenbock,BeerType_Wheatwine,BeerType_WinterWarmer,BeerType_Witbier,Gender_Female,Gender_Male,Gender_unknown
0,22,12300,2634,4.0,19,9,7.4,6,14,5,...,0,0,0,0,0,0,0,0,0,1
1,27,12300,5634,4.5,48,9,7.4,4,10,2,...,0,0,0,0,0,0,0,0,0,1
2,28,12300,3544,4.5,227,9,7.4,5,10,12,...,0,0,0,0,0,0,0,0,0,1
3,40,12300,6521,4.0,81,9,5.5,4,27,8,...,0,0,0,0,0,0,0,0,1,0
4,43,12300,10177,4.5,58,9,5.5,1,10,8,...,0,0,0,0,0,0,0,1,0,0


In [27]:
# free up the memory
del dfFullData
del dfFullDataVali

In [28]:
# Get all the columns
col_names = df_train_data.columns

idCols = ['RowID','BeerID','ReviewerID']
feature_cols =  col_names.drop(['RowID','BeerID','ReviewerID','rating' ])
target_col = 'rating'

# Create the sub data sets of the features and the target
dfTrainIds = df_train_data[idCols]
dfTrainFeatures = df_train_data[feature_cols]
dfTrainTarget = df_train_data[target_col]

dfValiIds = df_vali_data[idCols]
dfValiFeatures = df_vali_data[feature_cols]
dfValiTarget = df_vali_data[target_col]


In [29]:
print(dfTrainFeatures.shape)
dfTrainFeatures.head()

(746207, 2192)


,ReviewerReviewCount,BeerReviewCount,ABV,DayofWeek,DayofMonth,Month,Year,TimeOfDay,Birthday,BrewerID_1,...,BeerType_SmokedBeer,BeerType_Tripel,BeerType_ViennaLager,BeerType_Weizenbock,BeerType_Wheatwine,BeerType_WinterWarmer,BeerType_Witbier,Gender_Female,Gender_Male,Gender_unknown
0,200,23,7.4,1,23,5,2011,56188,0,0,...,0,0,0,0,0,0,0,0,1,0
1,10,23,7.4,1,16,5,2011,1906,0,0,...,0,0,0,0,0,0,0,0,1,0
2,164,23,7.4,7,10,4,2011,44246,0,0,...,0,0,0,0,0,0,0,0,0,1
3,432,23,7.4,3,30,3,2011,50880,0,0,...,0,0,0,0,0,0,0,0,1,0
4,500,23,7.4,4,24,3,2011,50820,37,0,...,0,0,0,0,0,0,0,0,1,0


In [30]:
def objective(trial):
  # Create the Light GBM Regression model and train
  model = lgb.LGBMRegressor(objective="regression_l1", metric="mae", random_state=seed
    ,learning_rate=trial.suggest_float("learning_rate", 0.0001, 0.5)
    ,num_leaves=trial.suggest_int("num_leaves", 2, 256)
    ,max_depth=trial.suggest_int("max_depth", 10, 50)
    ,n_estimators=trial.suggest_int("n_estimators ", 10, 1500)
    ,min_split_gain=trial.suggest_float("min_split_gain", 0.001, 1.0)
    ,min_child_samples=trial.suggest_int("min_child_samples", 1, 200)  
    #,min_child_weight =trial.suggest_float("min_child_weight", 0.0001, 0.1) 
    ,subsample =trial.suggest_float("subsample", 0.1, 1.0) 
    ,subsample_freq =trial.suggest_int("subsample_freq", 0, 15)
    ,colsample_bytree =trial.suggest_float("colsample_bytree", 0.1, 1.0) 
    ,reg_alpha =trial.suggest_float("reg_alpha", 0.1, 1.0) 
    ,reg_lambda =trial.suggest_float("reg_lambda", 0.1, 1.0)      
  )

  model.fit(X=dfTrainFeatures, y=dfTrainTarget)

  # Use the model to predict against our validation data
  test_predicted = model.predict(dfValiFeatures)  

  mae = mean_absolute_error(dfValiTarget, test_predicted)

  return mae

In [31]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)

print("\n---------")
print("Study Complete")
study.best_params

[I 2021-10-22 00:47:50,272] A new study created in memory with name: no-name-4c18e4da-e38d-4389-ae3d-3f3749be0f29
[I 2021-10-22 00:52:23,263] Trial 0 finished with value: 0.49214898020929204 and parameters: {'learning_rate': 0.45166758213274877, 'num_leaves': 217, 'max_depth': 10, 'n_estimators ': 847, 'min_split_gain': 0.6274441862709375, 'min_child_samples': 6, 'subsample': 0.8802453895679407, 'subsample_freq': 1, 'colsample_bytree': 0.2350892291464941, 'reg_alpha': 0.837261005274221, 'reg_lambda': 0.7919635329272813}. Best is trial 0 with value: 0.49214898020929204.
[I 2021-10-22 00:56:36,317] Trial 1 finished with value: 0.485829416088165 and parameters: {'learning_rate': 0.4767063433287476, 'num_leaves': 217, 'max_depth': 37, 'n_estimators ': 185, 'min_split_gain': 0.7747453296819841, 'min_child_samples': 57, 'subsample': 0.38736116794046915, 'subsample_freq': 6, 'colsample_bytree': 0.19020881620056648, 'reg_alpha': 0.21799725115770788, 'reg_lambda': 0.9542483963534442}. Best is t

In [ ]:
print(study.best_params)
print("Best Rank Score: " + str(study.best_value))
print("-------")
print(study.best_trial)

# Results

### Full Features - All Cols
* 50 trial of just top 4 params (4 min ish):
* {'learning_rate': 0.09075359977364383, 'num_leaves': 120, 'max_depth': 40, 'n_estimators ': 248, 'min_split_gain': 0.6310082232017945, 'min_child_samples': 35, 'subsample': 0.9466694477903548, 'subsample_freq': 0, 'colsample_bytree': 0.29392263338193186, 'reg_alpha': 0.891904482598078, 'reg_lambda': 0.4521335679885054}.
* Best is trial 23 with value: 0.46338874540827657


### All Cols
* 100 trial of just top 4 params (4 min ish):
* {'learning_rate': 0.012319449084467107, 'num_leaves': 117, 'max_depth': 22, 'n_estimators ': 295}
* Best Rank Score: 0.4454081741693529

### Beer Context
* 100 Trial of just the top 4 params (25 minutes):
* {'learning_rate': 0.010443500090385492, 'num_leaves': 68, 'max_depth': 14, 'n_estimators ': 608}
* Best Rank Score: 0.4450000625985502


### Consumer Context
* 100 Trial of just the top 4 params (25 minutes):
* {'learning_rate': 0.26879548049242075, 'num_leaves': 91, 'max_depth': 2, 'n_estimators ': 384}
* Best Rank Score: 0.4599019635783294